# IAIso LIVE Pressure Demo\n\nThis notebook runs the **canonical** `PressureEngine` and shows pressure rising, escalation, and release/reset.\n\n**No drift:** this uses the same engine as the server and CLI.

In [ ]:
import os, sys\nsys.path.insert(0, os.path.abspath('../..'))\nfrom iaiso_live.util.env import load_env\nfrom iaiso_live.core.pressure_engine import PressureEngine, PressureConfig\nfrom iaiso_live.core.events import EventSink\nfrom iaiso_live.adapters.simulated_llm import simulated_call\n\ncfg_raw = load_env('LIVE-TEST/config/live-test.env')\ncfg = PressureConfig(\n    pressure_threshold=float(cfg_raw.get('PRESSURE_THRESHOLD', 0.85)),\n    release_threshold=float(cfg_raw.get('RELEASE_THRESHOLD', 0.95)),\n    dissipation_rate=float(cfg_raw.get('DISSIPATION_RATE', 0.02)),\n    token_gain=float(cfg_raw.get('TOKEN_GAIN', 0.015)),\n    tool_gain=float(cfg_raw.get('TOOL_GAIN', 0.08)),\n)\nsink = EventSink(log_path='LIVE-TEST/docs/live-events.jsonl')\nengine = PressureEngine(cfg, sink=sink)

In [ ]:
pressures = []\nstatuses = []\nfor i in range(1, 25):\n    r = simulated_call(i)\n    status = engine.update(tokens=r['tokens'], tools=r['tools'])\n    pressures.append(engine.snapshot()['pressure'])\n    statuses.append(status)\n    if status == 'RELEASED':\n        break\npressures, statuses[-5:]

In [ ]:
import matplotlib.pyplot as plt\nplt.figure()\nplt.plot(range(1, len(pressures)+1), pressures)\nplt.xlabel('Step')\nplt.ylabel('Pressure')\nplt.title('IAIso LIVE: Pressure Over Time')\nplt.show()